In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,\
    accuracy_score, f1_score, precision_score, recall_score, classification_report
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50

from tqdm import tqdm

from datasets import ImageFolderWithSubfolders

precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

def get_dataloaders(dataset_dirs):

    train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    valid_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    train_dataset = ImageFolderWithSubfolders(root='../data/input/train',
                                              transform=train_transform,
                                              subfolders=dataset_dirs)
    valid_dataset = ImageFolderWithSubfolders(root='../data/input/valid',
                                              transform=valid_transform,
                                              subfolders=dataset_dirs)

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=64, shuffle=False)

    return train_dataloader, valid_dataloader

def preprocess_for_ssd( image):
        img = Image.fromarray(image)
        transform = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        img = transform(img)
        img = img.to(self.device)
        img = img.unsqueeze(0)
        return img    
 
def train_model(model, train_dataloader, valid_dataloader, start_epoch=0, num_epochs=5, plot=False):
    #model.cuda()
    #optimizer = optim.Adam(params=model.fc.parameters(), lr=0.0001, betas=(0.9, 0.999))
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    criterion = nn.CrossEntropyLoss()
    losses = []
    for epoch in range(start_epoch,num_epochs):
        print(epoch)
        model.train()
        for images, labels in tqdm(train_dataloader):
            optimizer.zero_grad()
            #images = images.cuda()
            #labels = labels.cuda()
            pre_img = preprocess_for_ssd(images)
            pred = model(pre_img)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        eval_model(model, valid_dataloader)
        #torch.save(model, '../checkpoints/text_classifier_{}.pt'.format(epoch+1))
        torch.save(model, '../checkpoints/text_classifier_2cats_{}.pt'.format(epoch+1))
        if plot:
            plt.plot(losses)

    return model
    
if __name__=='__main__':
    dataset_dirs = ['pearson', 'no_pearson']
    train_dataloader, valid_dataloader = get_dataloaders(dataset_dirs)
    model = ssd_model
    model = train_model(model, train_dataloader, valid_dataloader, start_epoch=0, num_epochs=10)


Using cache found in C:\Users\komleva-ep/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
  0%|          | 0/4 [00:00<?, ?it/s]

0


  0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute '__array_interface__'